<a href="https://colab.research.google.com/github/ewu2023/CS589-HW4/blob/main/CS589_NeuralNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [9]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Define Neural Network Class

In [88]:
class NeuralNetwork():
    def __init__(self, networkShape, trainData: pd.DataFrame, weights=None):
        """
        Constructor for the neural network class.
        
        networkShape: A list of integers that contains the number of neurons to use in each layer

        trainData: The data set that will be used to train the model

        weights: A list of weight matrices for each hidden layer. If initialized to None, the constructor will assign random weights
        """
        self.trainData = trainData

        # Get a copy of the training data without the labels
        self.noLabelTrainData = trainData.loc[:, trainData.columns!='class']

        # Instance variable storing weight matrices
        self.layers = []
        
        # Initialize layers
        for i in range(len(networkShape) - 1):
            # Get the number of neurons in the current and next layers
            numCurLayer = networkShape[i] + 1 # Account for neurons + bias term in current layer
            numNextLayer = networkShape[i + 1]

            # Initialize matrix for the current layer
            # Number of rows = number of neurons in layer i + 1
            # Number of columns = number of neurons in layer i
            layerMatrix = np.zeros(shape=(numNextLayer, numCurLayer))
            if weights:
                layerMatrix = weights[i]
            else:
                self._init_matrix(layerMatrix)
            
            # Append current layer to the list of layers
            self.layers.append(layerMatrix)
        
    def _init_matrix(self, matrix: np.ndarray):
        rows, cols = matrix.shape
        for i in range(rows):
            for j in range(cols):
                matrix[i, j] = norm.rvs()
    
    # Definition for the sigmoid function
    def sigmoid(self, x):
        return (1 / (1 + np.exp(-x)))
    
    # Compute activation vector
    def compute_activation_vector(self, weightedSums: np.ndarray):
        numRows, numCols = weightedSums.shape
        activationVector = np.zeros(shape=(numRows, numCols))

        for i in range(numRows):
            # Get weighted sum from i-th row
            x = weightedSums[i, 0]

            # Compute output of sigmoid function and place it in activation vector
            activationVector[i, 0] = self.sigmoid(x)
        
        return activationVector
    
    # Method for propagating forward one instance
    def propagate_one(self, instance):
        # Add a bias term to the instance
        instanceAsNP = instance.to_numpy()
        instanceVector = np.concatenate(([1], instanceAsNP))
        
        # Make instance vector a column vector
        instanceVector = np.atleast_2d(instanceVector).T
        
        # Iterate over each layer and compute activations for each neuron
        prevActivation = instanceVector # Keep track of the activation vector for previous layer
        for i in range(len(self.layers) - 1):
            # Get current weight matrix
            curTheta = self.layers[i]

            # Compute weighted sum vector (z-matrix): Theta^{l=i-1} * a^{l=i-1}
            z = np.matmul(curTheta, prevActivation)

            # Compute activation vector of current layer
            curActivationVec = self.compute_activation_vector(z)
            
            # Add bias term to current activation vector
            curActivationVec = np.concatenate(([[1]], curActivationVec)) # Prepend 1 to vector

            # Update previous activation vector
            prevActivation = curActivationVec
        
        # Compute activation at the final layer
        lastTheta = self.layers[len(self.layers) - 1]
        lastZMat = np.matmul(lastTheta, prevActivation)
        outputVector = self.compute_activation_vector(lastZMat)

        return outputVector[0, 0] # Need to do this because output vector is a column vector



Test Neural Net

In [89]:
d = {
    'x': [0.13000, 0.42000], 
    'class': [0.90000, 0.23000]
}

df = pd.DataFrame(data=d)
df_noLabels = df.loc[:, df.columns!='class']

networkShape = [1, 2, 1]
weights = [
    np.array(
        [[0.40000, 0.10000],
         [0.30000, 0.20000]],
    ),

    np.array([[0.70000, 0.50000, 0.60000]])
]

network = NeuralNetwork(networkShape, trainData=df, weights=weights)
curInstance = df_noLabels.iloc[0]
print(network.propagate_one(curInstance))

[[0.79402743]]


# Test Code

In [45]:
testData = {'col1': [1, 2], 'col2': [3, 4], 'class': ['yes', 'no']}
testFrame = pd.DataFrame(data=testData)

for i in range(len(testFrame)):
    row = testFrame.iloc[i]
    print(row)

col1       1
col2       3
class    yes
Name: 0, dtype: object
col1      2
col2      4
class    no
Name: 1, dtype: object
